<a href="https://colab.research.google.com/github/yoonkim313/dataCampusProject-Team10/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
p = [i for i in text.split("\n") if i]
d = [kss.split_sentences(i) for i in p if i]
print(d)
rel = Relation()
for ds in d:
  for i in range(len(ds)):
    print(rel.relationPopup(ds[i]))


[['평등은 자유와 더불어 근대 사회의 핵심 이념으로 자리 잡고있다.', '인간은 가령 인종이나 성별과 상관없이 누구나 평등하다고 생각한다.', '모든 인간은 평등하다고 말하는데, 이 말은 무슨 뜻일까?', '그리고 그 근거는 무엇인가?', '일단 이 말을 모든 인간을 모든 측면에서 똑같이 대우하는 절대적 평등으로 생각하는 이는 없다.', '인간은 저마다 다르게 가지고 태어난 능력과 소질을 똑같게 만들 수 없기 때문이다.', '절대적 평등은 개인의 개성이냐 자율성 등의 가치와 충돌하기도 한다.'], ['평등에 대한 요구는 모든 불평등을 악으로 보는 것이 아니라 충분한 이유가 제시되지 않은 불평등을 제거하는 데 목표를 두고 있다.', "'이유 없는 차별 금지'라는 조건적 평등 원칙은 차별 대우를 할 때는 이유를 제시할 것을 요구하고 있다.", '이것은 어떤 이유가 제시된다면 특정한 부류에 속하는 사람들에게는 평등한 대우를, 그 부류에 속하지 않는 사람들에게는 차별적 대우를 하는 것을 허용한다.', '그렇다면 사람들을 특정한 부류로 구분하는 기준은 무엇인가?', '이것은 바로 평등의 근거에 대한 물음이다.'], ['근대의 여러 인권 선언에 나타난 평등 개념은 개인들 사이의 평등성을 타고난 자연적 권리로 간주하였다.', '하지만 이러한 자연권 이론은 무엇이 자연적 권리이고 권리의 존재가 자명한 이유가 무엇인지 등의 문제에 부딪히게 된다.', '그래서 롤스는 기존의 자연권 사상에 의존하지 않는 방시으로 인간 평등의 근거를 마런하려고 한다.', '그는 어떤 규칙이 공평하고 일관되게 운영되며, 그 규칙에 따라 유사한 경우는 유사하게 취급된다면 형식적 정의는 실현된다고 본다.', '하지만 롤스는 형식적 정의에 따라 규칙을 준수하는 것만으로는 정의를 담보할 수 없다고 생각한다.', '그 규칙이 더 높은 도덕적 권위를 지닌 다른 이념과 충돌할 수 었기에, 실질적 정의가 보장되기 위해서는 규칙의 내용이 중요한 것이다.'], ['롤스는 인간 평등의 근거를 설명하면서 영역 성질 (

../frameBERT/src/utils.py:279: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred_logits = sm(masked_logit).view(1,-1)


 Parsed Candidate  [['평등은', '자유와', '더불어', '근대', '사회의', '핵심', '이념으로', '자리', '잡고있다.'], ['_', '자유.n', '_', '_', '_', '_', '_', '_', '_'], ['_', 'Strictness', '_', '_', '_', '_', '_', '_', '_'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]
 Parsed Candidate  [['평등은', '자유와', '더불어', '근대', '사회의', '핵심', '이념으로', '자리', '잡고있다.'], ['_', '_', '_', '_', '사회.n', '_', '_', '_', '_'], ['_', '_', '_', '_', 'Aggregate', '_', '_', '_', '_'], ['O', 'O', 'O', 'B-Type', 'O', 'O', 'O', 'O', 'O']]
 Parsed Candidate  [['평등은', '자유와', '더불어', '근대', '사회의', '핵심', '이념으로', '자리', '잡고있다.'], ['_', '_', '_', '_', '_', '핵심.n', '_', '_', '_'], ['_', '_', '_', '_', '_', 'Importance', '_', '_', '_'], ['O', 'O', 'O', 'O', 'O', 'O', 'B-Factor', 'O', 'O']]
 Parsed Candidate  [['평등은', '자유와', '더불어', '근대', '사회의', '핵심', '이념으로', '자리', '잡고있다.'], ['_', '_', '_', '_', '_', '_', '_', '자리.n', '_'], ['_', '_', '_', '_', '_', '_', '_', 'Locale', '_'], ['O', 'O', 'O', 'B-Stimulus', 'I-Entity', 'I-Entity', 'I-Entity', 'O', 'O']]
 Parsed Can



> 공유 드라이브와 작업 환경 연결



In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
nb_path = '/content/notebook'
os.symlink('/content/drive/Shared drives/BigDATA TEAM 10', nb_path)
sys.path.insert(0,nb_path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive




> 사전에 설치된 라이브러리의 목록


    !pip install --target=$nb_path transformers
    !apt-get update
    !apt-get g++ openjdk-8-jdk 
    !pip3 install --target=$nb_path konlpy
    !pip install --target=$nb_path soykeyword
    !pip install --target=$nb_path krwordrank
    !pip install --target=$nb_path heapq
    !pip install --target=$nb_path kss
    !pip install --target=$nb_path bert
    !pip install --target=$nb_path textrankr
    !pip install --target=$nb_path lexrankr

In [181]:
with open("/content/test_text.txt",'r') as file:
  string = file.readlines()
print(string)
string = ' '.join(string)
print(string)
r = string.split("\n")
print(''.join(r))
# print(len(r))

['평등은 자유와 더불어 근대 사회의 핵심 이념으로 자리 잡고있다. 인간은 가령 인종이나 성별과\n', '상관없이 누구나 평등하다고 생각한다. 모든 인간은 평등하다고 말하는데, 이 말은 무슨 뜻일까?\n', '그리고 그 근거는 무엇인가? 일단 이 말을 모든 인간을 모든 측면에서 똑같이 대우하는 절대적\n', '평등으로 생각하는 이는 없다. 인간은 저마다 다르게 가지고 태어난 능력과 소질을 똑같게 만들\n', '수 없기 때문이다. 절대적 평등은 개인의 개성이냐 자율성 등의 가치와 충돌하기도 한다.\n', '평등에 대한 요구는 모든 불평등을 악으로 보는 것이 아니라 충분한 이유가 제시되지 않은 불평\n', "등을 제거하는 데 목표를 두고 있다. '이유 없는 차별 금지'라는 조건적 평등 원칙은 차별 대우를\n", '할 때는 이유를 제시할 것을 요구하고 있다. 이것은 어떤 이유가 제시된다면 특정한 부류에 속하\n', '는 사람들에게는 평등한 대우를, 그 부류에 속하지 않는 사람들에게는 차별적 대우를 하는 것을\n', '허용한다. 그렇다면 사람들을 특정한 부류로 구분하는 기준은 무엇인가? 이것은 바로 평등의 근\n', '거에 대한 물음이다.\n', '근대의 여러 인권 선언에 나타난 평등 개념은 개인들 사이의 평등성을 타고난 자연적 권리로 간\n', '주하였다. 하지만 이러한 자연권 이론은 무엇이 자연적 권리이고 권리의 존재가 자명한 이유가\n', '무엇인지 등의 문제에 부딪히게 된다. 그래서 롤스는 기존의 자연권 사상에 의존하지 않는 방시\n', '으로 인간 평등의 근거를 마런하려고 한다. 그는 어떤 규칙이 공평하고 일관되게 운영되며, 그 규\n', '칙에 따라 유사한 경우는 유사하게 취급된다면 형식적 정의는 실현된다고 본다. 하지만 롤스는\n', '형식적 정의에 따라 규칙을 준수하는 것만으로는 정의를 담보할 수 없다고 생각한다. 그 규칙이\n', '더 높은 도덕적 권위를 지닌 다른 이념과 충돌할 수 었기에, 실질적 정의가 보장되기 위해서는\n', '규칙의 내용이 중요

In [182]:
from konlpy.tag import Hannanum, Okt
import re
import time
import numpy as np
import heapq
import pandas as pd
from operator import itemgetter
from collections import deque, defaultdict
from ast import literal_eval
from collections import defaultdict
from soykeyword.lasso import LassoKeywordExtractor
from pprint import pprint
from krwordrank.word import KRWordRank
from copy import deepcopy
import kss
import itertools
import unicodedata
import requests
from functools import reduce
from transformers import *
import torch
from bs4 import BeautifulSoup
import string
from textrankr import TextRank
from lexrankr import LexRank

In [183]:
%cd /content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction/frameBERT
import frame_parser
path="/content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction"
parser = frame_parser.FrameParser(model_path=path, language='ko')
h = Hannanum()
okt = Okt()

/content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction/frameBERT
srl model: framenet
language: ko
version: 1.2
using viterbi: False
using masking: True
pretrained BERT: bert-base-multilingual-cased
using TGT special token: True
used dictionary:
	 /content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction/frameBERT/src/../koreanframenet/resource/info/kfn1.2_lu2idx.json
	 /content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction/frameBERT/src/../koreanframenet/resource/info/kfn1.2_lufrmap.json
	 /content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction/frameBERT/src/../koreanframenet/resource/info/mul_bio_frargmap.json
...loaded model path: /content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction
/content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction
...model is loaded


#### Linked List 구현해서 parsed 후보군에 사용

In [ ]:
class Node:
    def __init__(self, data):
        self.words = data[0]
        self.usage = data[1]
        self.tags = data[3]
        self.next = None

    def __repr__(self):
        return str((self.words, self.tags))

class LinkedList:
  def __init__(self):
      self.head = None

  def __repr__(self):
      node = self.head
      nodes = []
      while node is not None:
          nodes.append(str(node.tags))
          node = node.next
      nodes.append("None")
      return ' -> '.join(nodes)


### Multiple Inheritance by Super()

  **Text --> Highlight --> Relation**


Text 클래스

    주어진 텍스트를 문단별로, 문장별로 나누어줌

Highlight 클래스
     
     CSS 문법을 사용하여 중심문장에는 underline, 중심 단어에는 highlight, 관계성을 나타내는 단어들에는 box를 삽입해준다

Relation 클래스

    frameNET의 807개의 의미역을 사용하여 단어들의 다의성을 고려한 용례, beginning/ inside/ outside tagging을 이용한다. 구절 사이의 언어적 관계성을 파악할 수 있으며 이를 통해 요약, 압축을 구현함.
     

  Relation.__mro__
  result : (__main__.Relation, __main__.Highlight, __main__.Text, object)

In [79]:
text = '''
평등은 자유와 더불어 근대 사회의 핵심 이념으로 자리 잡고있다. 인간은 가령 인종이나 성별과 상관없이 누구나 평등하다고 생각한다. 모든 인간은 평등하다고 말하는데, 이 말은 무슨 뜻일까? 그리고 그 근거는 무엇인가? 일단 이 말을 모든 인간을 모든 측면에서 똑같이 대우하는 절대적 평등으로 생각하는 이는 없다. 인간은 저마다 다르게 가지고 태어난 능력과 소질을 똑같게 만들 수 없기 때문이다. 절대적 평등은 개인의 개성이냐 자율성 등의 가치와 충돌하기도 한다. 
평등에 대한 요구는 모든 불평등을 악으로 보는 것이 아니라 충분한 이유가 제시되지 않은 불평등을 제거하는 데 목표를 두고 있다. '이유 없는 차별 금지'라는 조건적 평등 원칙은 차별 대우를 할 때는 이유를 제시할 것을 요구하고 있다. 이것은 어떤 이유가 제시된다면 특정한 부류에 속하는 사람들에게는 평등한 대우를, 그 부류에 속하지 않는 사람들에게는 차별적 대우를 하는 것을 허용한다. 그렇다면 사람들을 특정한 부류로 구분하는 기준은 무엇인가? 이것은 바로 평등의 근거에 대한 물음이다.
근대의 여러 인권 선언에 나타난 평등 개념은 개인들 사이의 평등성을 타고난 자연적 권리로 간주하였다. 하지만 이러한 자연권 이론은 무엇이 자연적 권리이고 권리의 존재가 자명한 이유가 무엇인지 등의 문제에 부딪히게 된다. 그래서 롤스는 기존의 자연권 사상에 의존하지 않는 방시으로 인간 평등의 근거를 마런하려고 한다. 그는 어떤 규칙이 공평하고 일관되게 운영되며, 그 규칙에 따라 유사한 경우는 유사하게 취급된다면 형식적 정의는 실현된다고 본다. 하지만 롤스는 형식적 정의에 따라 규칙을 준수하는 것만으로는 정의를 담보할 수 없다고 생각한다. 그 규칙이 더 높은 도덕적 권위를 지닌 다른 이념과 충돌할 수 었기에, 실질적 정의가 보장되기 위해서는 규칙의 내용이 중요한 것이다.
롤스는 인간 평등의 근거를 설명하면서 영역 성질 (range property) 개념을 도입한다. 예를 들어 어떤 원의 내부에 있는 점들은 그 위치가 서로 다르지만 원의 내부에 있다는 점에서 동일한 영역 성질을 갖는다. 반면에 원의 내부에 있는 점과 원의 외부에 있는 점은 원의 경계선을 기준으로 서로 다른 영역 성질을 갖는다. 그는 평등한 대우를 받기 위한 영역 성질로서 '도덕적 인격'을 제시한다. 도덕적 인격이란 도덕적 호소가 가능하고 그런 호소에 관심을 가지는 능력이 있다는 것인데, 이 능력을 최소치만 갖고 있다면 평등한 대우에 대한 권한을 갖게 된다. 도덕적 인격이라고 해서 도덕적으로 훌륭하다는 뜻이 아니라 도덕과 무관하다는 말과 대비되는 뜻으로 쓰고 있다. 그런데 어린 아이는 인격체로서의 최소한의 기준을 충족하고 있는지가 논란이 될 수 있다. 이에 대해 롤스는 도덕적 인격을 규정하는 최소한의 요구 조건은 잠재적 능력이지 그것의 실현 여부가 아니기에 어린 아이도 평등한 존재라고 말한다. 
'''

In [100]:
class Text():
    def __init__(self, text):
        text = re.sub("'", ' ', text)
        paragraphs = text.split('\n')
        self.text = text
        self.paragraphs = [i for i in paragraphs if i]
        self.counts = len(self.paragraphs)
        self.docs = [kss.split_sentences(paragraph) for paragraph in paragraphs if kss.split_sentences(paragraph)]
        self.newtext = deepcopy(self.text)
        print("TEXT")

    def findall(self, p, s):
        i = s.find(p)
        while i != -1:
            yield i
            i = s.find(p, i + 1)
      
    def countMatcher(self, sentences, paragraph_no):
        paragraph = self.docs[paragraph_no]
        total_no = len(paragraph)
        vec = [0] * total_no
        
        for idx, candidate in enumerate(paragraph):
            for sentence in sentences:
                if sentence[:4] in candidate:
                    vec[idx] += 1
        print("Vec ",vec)
        return vec

class Highlight(Text):
    def __init__(self, text):
        super().__init__(text)
        print("Highlight")

        wordrank_extractor = KRWordRank(min_count=4, max_length=10)
        self.keywords, rank, graph = wordrank_extractor.extract(self.paragraphs, num_keywords=20)
        self.path = "/content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction"
        p = []
        kw = []
        for k, v in self.keywords.items():
            p.append(okt.pos(k))
            kw.append(k)
        words = self.text.split(' ')
        s = set()
        keylist = [word for i in kw for word in words if i in word]
        keylist = [i for i in keylist if len(i)>2]
        for i in keylist:
            if len(i)>2:
              s.add(i)
        print("KEYLIST: ",keylist)
        p = [okt.pos(word) for word in s]
        s = set()
        for idx in range(len(p)):
            ls = p[idx]
            for j in range(len(ls)):
                tag = ls[j][1]
                word = ls[j][0]
                if tag == "Noun":
                    s.add(word)
        self.keys = []
        for temp in s:
            self.keys.append(" " + temp)

    
    def add_tags_keywords(self):
        self.candidates = self.keys
        self.newtext = deepcopy(self.text)
        self.idx = [(i, i + len(candidate)) for candidate in self.candidates for i in
                        self.findall(candidate, self.text)]
        for i in range(len(self.idx)):
            try:
                self.idx = [(start, start + len(candidate)) for candidate in self.candidates for start in
                            self.findall(candidate, self.newtext)]
                word = self.newtext[self.idx[i][0]:self.idx[i][1]]
                tagged = " <mark style='background-color:#FFD0F2'>%s</mark>" % (word)
                self.newtext = tagged.join([self.newtext[:self.idx[i][0]], self.newtext[self.idx[i][1]:]])
            except:
                pass
        return self.newtext


    def add_tags_conj(self, txt):
        conj = '그리고, 그런데, 그러나, 그래도, 그래서, 또는, 및, 즉, 게다가, 따라서, 때문에, 아니면, 왜냐하면, 단, 오히려, 비록, 예를 들어, 반면에, 하지만, 그렇다면, 바로, 이에 대해'
        conj = conj.replace("'", "")
        self.candidates = conj.split(",")
        self.newtext = deepcopy(txt)
        self.idx = [(i, i + len(candidate)) for candidate in self.candidates for i in
                        self.findall(candidate, txt)]
        for i in range(len(self.idx)):
            try:
                self.idx = [(start, start + len(candidate)) for candidate in self.candidates for start in
                            self.findall(candidate, self.newtext)]
                word = self.newtext[self.idx[i][0]:self.idx[i][1]]
                res = ""
                tagged = " <mark style='background-color:#F9D877'>%s</mark>" % (word)
                print(tagged)
                self.newtext = tagged.join([self.newtext[:self.idx[i][0]], self.newtext[self.idx[i][1]:]])
            except:
                pass
        return self.newtext

    def highlight(self):
          result = self.add_tags_keywords()
          cleanr = re.compile("</mark>.{0,5}<mark style='background-color:#FFD0F2'>")
          patterns = re.findall("</mark>.{0,5}<mark style='background-color:#FFD0F2'>",result)
          w = [i[7:-39] for i in patterns]
          for i in range(len(w)):
            result = re.sub(patterns[i], w[i], result)
          txt = self.add_tags_conj(result)
          print("Highlighted Result ", txt)
          return txt

class Relation(Highlight):
    def __init__(self):
      super().__init__(text)
      
    def frameParse(self, id):
        parser = frame_parser.FrameParser(model_path=self.path, language='ko')
        ps = parser.parser(self.docs[id], sent_id='1', result_format='conll')
        return ps

    def extractFrame(self):
        self.final = {}
        for paragraph in self.docs:
            print("PARAGRAPH: ", self.docs)
            for idx in range(len(paragraph)):
                parsed = frameParse(paragraph[idx])  # candidates 생성
                self.final.setdefault(idx, str)
                parsedList = LinkedList()
                for j in range(len(parsed)):
                    parsed_candidate = parsed[j]
                    new_node = Node(parsed_candidate)
                    if j == 0:
                        old_node = new_node
                        parsedList.head = old_node
                    elif j == len(parsed) - 1:
                        old_node.next = new_node
                        new_node.next = None
                        print(idx, '  ', parsedList)
                        self.final[idx] = parsedList
                    else:
                        old_node.next = new_node
                        old_node = new_node

    def findConsecutiveBIO(self, words, tag):
        began = False
        count = 1
        self.que = deque(words)
        for i in range(len(words)):
            if tag[i] == 'O' and not began:
                self.que.popleft()
                began = False
            if tag[i] == 'O' and began:
                self.que.pop()
                began = False
            if tag[i].startswith('B'):
                began = True
            if began & tag[i].startswith('I'):
                count += 1
        return self.que

    def extractRelation(self):
        s = ""
        self.res = [] * 20
        for k, v in self.final.items():
            a = v.head
            try:
                while a is not None:
                    i = 0
                    temp = " "
                    if len(a.words) == len(a.tags):
                        q = findConsecutiveBIO(a.words, a.tags)
                        print(f"The que {q}")
                    else:
                        print("ERROR OCCURED")

                    count = len(a.words)
                    for tag in a.tags:
                        if tag.startswith("O"):
                            count -= 1
                    if count > len(a.words) * 0.45:
                        print(f"The threshold is reached !!😏 The count is {count}")
                        temp = " ".join(q)
                        words = a.words
                        tags = a.tags
                    else:
                        temp = None
                        words = None
                        tags = None
                    self.res[i] = (words, tag, temp)
                    i+=1
                    a = a.next
            except:
                pass
        self.res = [j for j in self.res if j]

    def roles(self):
        for k, tup in self.res.items():
            a, b, c = tup
            roles = [0] * len(b)
            words = [0] * len(b)
            for i in range(len(b)):
                try:
                    roles[i] = b[i].split("-")[1]
                except:
                    roles[i] = b[i].split("-")[0]
            print(roles)
            for _ in roles:
                pass

class Summarize(Highlight):
    def __init__(self, text, paragraph_no):
      super().__init__(text)
      print("length of paragraphs ",len(self.paragraphs))
      self.txt = self.paragraphs[paragraph_no]
      self.paragraph_no = paragraph_no

    def summarize(self):
        url = "https://api.smrzr.io/summarize?ratio=0.15"
        headers = {
            'content-type': 'raw/text',
            'origin': 'https://smrzr.io',
            'referer': 'https://smrzr.io/',
            'sec-fetch-dest': 'empty',
            'sec-fetch-mode': 'cors',
            'sec-fetch-site': 'same-site',
            "user-agent": "Mozilla/5.0"
        }
        resp = requests.post(url, headers=headers, data= self.txt.encode('utf-8'))
        assert resp.status_code == 200
        summary = resp.json()['summary']
        temp = summary.split('\n')
        print("BERT: ", temp)
        return temp


    def summarizeTextRank(self, max=1):
        tr = TextRank(self.txt)
        
        summary = tr.summarize(max).split('\n')
        print("Textrank: ",summary)
        return summary


    def summarizeLexRank(self):
        lr = LexRank()
        lr.summarize(self.txt)
        summaries = lr.probe()
        print("Lexrank: ",summaries)
        return summaries

    def ensembleSummarize(self):
        a = np.array(self.countMatcher(self.summarize(), self.paragraph_no))
        
        try:
          b = np.array(self.countMatcher(self.summarizeLexRank(), self.paragraph_no))
        except:
          b = np.zeros_like(a)
        c = np.array(self.countMatcher(self.summarizeTextRank(),self.paragraph_no))
        result= a+b+c
        i, = np.where(result == max(result))
        txt, index = self.docs[self.paragraph_no][i[0]], i[0]
        return txt, index


### Hightlight && Underline 완료 
본문에 대한 attention catcher로 제공됨


평등은 자유와 더불어 근대 사회의 핵심 이념으로 자리 잡고있다.<mark style='background-color:#FFD0F2'> 인간</mark>은 가령 인종이나 성별과 상관없이 누구나 <mark style='background-color:#FFD0F2'> 평등</mark>하다고 생각한다.<u data-toggle='tooltip' data-placement='top' title=To be addedstyle='text-decoration:underline; text-decoration-color:#906fa8; font-weight: bold; text-decoration-style: wavy'>모든 <mark style='background-color:#FFD0F2'> 인간은  평등</mark>하다고 말하는데, 이 말은 무슨 뜻일까?</u><mark style='background-color:#F9D877'>그리고</mark> 그 <mark style='background-color:#FFD0F2'> 근거는  무엇</mark>인가?<u data-toggle='tooltip' data-placement='top' title=To be addedstyle='text-decoration:underline; text-decoration-color:#906fa8; font-weight: bold; text-decoration-style: wavy'>일단 이 말을 모든 <mark style='background-color:#FFD0F2'> 인간</mark>을 모든 측면에서 똑같이 <mark style='background-color:#FFD0F2'> 대우</mark>하는 절대적 <mark style='background-color:#FFD0F2'> 평등</mark>으로 생각하는 이는 없다.</u><mark style='background-color:#FFD0F2'> 인간</mark>은 저마다 다르게 가지고 태어난 <mark style='background-color:#FFD0F2'> 능력</mark>과 소질을 똑같게 만들 수 없기 때문이다.절대적 <mark style='background-color:#FFD0F2'> 평등</mark>은 개인의 개성이냐 자율성 등의 가치와 충돌하기도 한다.
<u data-toggle='tooltip' data-placement='top' title=To be addedstyle='text-decoration:underline; text-decoration-color:#906fa8; font-weight: bold; text-decoration-style: wavy'>평등에 대한 요구는 모든 <mark style='background-color:#FFD0F2'> 불평등</mark>을 악으로 보는 <mark style='background-color:#FFD0F2'> 것</mark>이 아니라 충분한 <mark style='background-color:#FFD0F2'> 이유가  제시</mark>되지 않은 <mark style='background-color:#FFD0F2'> 불평등</mark>을 제거하는 데 목표를 두고 있다.</u><mark style='background-color:#FFD0F2'> 이유</mark> 없는 차별 금지 라는 조건적 <mark style='background-color:#FFD0F2'> 평등</mark> 원칙은 차별 <mark style='background-color:#FFD0F2'> 대우</mark>를 할 때는 <mark style='background-color:#FFD0F2'> 이유를  제시할  것</mark>을 요구하고 있다.이것은 어떤 <mark style='background-color:#FFD0F2'> 이유가  제시</mark>된다면 특정한 부류에 속하는 사람들에게는 <mark style='background-color:#FFD0F2'> 평등한  대우</mark>를, 그 부류에 속하지 않는 사람들에게는 차별적 <mark style='background-color:#FFD0F2'> 대우를 하는  것</mark>을 허용한다.<mark style='background-color:#F9D877'> 그렇다면</mark> 사람들을 특정한 부류로 구분하는 기준은 <mark style='background-color:#FFD0F2'> 무엇</mark>인가?이것은 <mark style='background-color:#F9D877'> 바로</mark> <mark style='background-color:#FFD0F2'> 평등의  근거</mark>에 대한 물음이다.
근대의 여러 인권 선언에 나타난 <mark style='background-color:#FFD0F2'> 평등</mark> 개념은 개인들 사이의 <mark style='background-color:#FFD0F2'> 평등</mark>성을 타고난 <mark style='background-color:#FFD0F2'> 자연</mark>적 권리로 간주하였다.<u data-toggle='tooltip' data-placement='top' title=To be addedstyle='text-decoration:underline; text-decoration-color:#906fa8; font-weight: bold; text-decoration-style: wavy'><mark style='background-color:#F9D877'> 하지만</mark> 이러한 <mark style='background-color:#FFD0F2'> 자연</mark>권 이론은 <mark style='background-color:#FFD0F2'> 무엇이  자연</mark>적 권리이고 권리의 존재가 자명한 <mark style='background-color:#FFD0F2'> 이유가  무엇</mark>인지 등의 문제에 부딪히게 된다.</u><mark style='background-color:#F9D877'> 그래서</mark> <mark style='background-color:#FFD0F2'> 롤스</mark>는 기존의 <mark style='background-color:#FFD0F2'> 자연</mark>권 사상에 의존하지 않는 방시으로 <mark style='background-color:#FFD0F2'> 인간  평등의  근거</mark>를 마런하려고 한다.그는 어떤 <mark style='background-color:#FFD0F2'> 규칙</mark>이 공평하고 일관되게 운영되며, 그 <mark style='background-color:#FFD0F2'> 규칙</mark>에 따라 유사한 경우는 유사하게 취급된다면 형식적 <mark style='background-color:#FFD0F2'> 정의</mark>는 실현된다고 본다.<u data-toggle='tooltip' data-placement='top' title=To be addedstyle='text-decoration:underline; text-decoration-color:#906fa8; font-weight: bold; text-decoration-style: wavy'><mark style='background-color:#F9D877'> 하지만</mark> <mark style='background-color:#FFD0F2'> 롤스</mark>는 형식적 <mark style='background-color:#FFD0F2'> 정의에 따라  규칙</mark>을 준수하는 <mark style='background-color:#FFD0F2'> 것만으로는  정의</mark>를 담보할 수 없다고 생각한다.</u>그 <mark style='background-color:#FFD0F2'> 규칙</mark>이 더 높은 <mark style='background-color:#FFD0F2'> 도덕</mark>적 권위를 지닌 다른 이념과 충돌할 수 었기에, 실질적 <mark style='background-color:#FFD0F2'> 정의</mark>가 보장되기 위해서는 <mark style='background-color:#FFD0F2'> 규칙</mark>의 내용이 중요한 <mark style='background-color:#FFD0F2'> 것</mark>이다.
롤스는 <mark style='background-color:#FFD0F2'> 인간  평등의  근거</mark>를 설명하면서 영역 <mark style='background-color:#FFD0F2'> 성질</mark> (range property) 개념을 도입한다.<mark style='background-color:#F9D877'> 예를 들어</mark> 어떤 원의 내부에 있는 점들은 그 위치가 서로 다르지만 원의 내부에 있다는 점에서 동일한 영역 <mark style='background-color:#FFD0F2'> 성질</mark>을 갖는다.<mark style='background-color:#F9D877'> 반면에</mark> 원의 내부에 있는 점과 원의 외부에 있는 점은 원의 경계선을 기준으로 서로 다른 영역 <mark style='background-color:#FFD0F2'> 성질</mark>을 갖는다.<u data-toggle='tooltip' data-placement='top' title=To be addedstyle='text-decoration:underline; text-decoration-color:#906fa8; font-weight: bold; text-decoration-style: wavy'>그는 <mark style='background-color:#FFD0F2'> 평등한  대우</mark>를 받기 위한 영역 <mark style='background-color:#FFD0F2'> 성질로서   도덕적  인격 을  제시한다.</u>도덕적  인격이란  도덕</mark>적 호소가 가능하고 그런 호소에 관심을 가지는 <mark style='background-color:#FFD0F2'> 능력</mark>이 있다는 <mark style='background-color:#FFD0F2'> 것</mark>인데, 이 <mark style='background-color:#FFD0F2'> 능력</mark>을 최소치만 갖고 있다면 <mark style='background-color:#FFD0F2'> 평등한  대우</mark>에 대한 권한을 갖게 된다.<mark style='background-color:#FFD0F2'> 도덕적  인격</mark>이라고 해서 <mark style='background-color:#FFD0F2'> 도덕</mark>적으로 훌륭하다는 뜻이 아니라 <mark style='background-color:#FFD0F2'> 도덕</mark>과 무관하다는 말과 대비되는 뜻으로 쓰고 있다.<mark style='background-color:#F9D877'> 그런데</mark> 어린 아이는 <mark style='background-color:#FFD0F2'> 인격</mark>체로서의 최소한의 기준을 충족하고 있는지가 논란이 될 수 있다.<mark style='background-color:#F9D877'> 이에 대해</mark> <mark style='background-color:#FFD0F2'> 롤스는  도덕적  인격</mark>을 규정하는 최소한의 요구 조건은 잠재적 <mark style='background-color:#FFD0F2'> 능력</mark>이지 그것의 실현 여부가 아니기에 어린 아이도 <mark style='background-color:#FFD0F2'> 평등</mark>한 존재라고 말한다.

### Relational Information 추가 정보를 팝업으로 제공

In [270]:
class Relation(object):
  def __init__(self):
      self.path="/content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction"
      self.parser = parser

  def findConsecutiveBIO(self, words, tag):
      began = False
      count = 1
      que = " "
      temp_word = []
      temp_tag = []
      res = " "

      for i in range(len(words)):
          if not tag[i].startswith("O"):
              began = True
              temp_word.append(words[i])
              temp_tag.append(tag[i].split("-")[1])
          elif tag[i].startswith("O"):
              began = False
      r = que.join(temp_word)
      s = que.join(temp_tag)
      return r, s

  def relationPopup(self, sent):
      parsed = parser.parser(sent, sent_id='1', result_format='conll')
      parsedList = LinkedList()
      self.final = dict()
      for j in range(len(parsed)):
          parsed_candidate = parsed[j]
          print(" Parsed Candidate ", parsed_candidate)
          new_node = Node(parsed_candidate)
          if j == 0:
              old_node = new_node
              parsedList.head = old_node
          elif j == len(parsed) - 1:
              old_node.next = new_node
              new_node.next = None
              print(idx, '  ', parsedList)
              self.final[idx] = parsedList
          else:
              old_node.next = new_node
              old_node = new_node
      res = dict()

      for k, v in self.final.items():
          a = v.head
          while a is not None:
              i = 0
              temp = " "
              if len(a.words) == len(a.tags):
                  w, t= self.findConsecutiveBIO(a.words, a.tags)
                  res[t]=w
              else:

                  print("ERROR OCCURED")
              a = a.next

      return res
      
  def arrow(self, words, tags):
      d = defaultdict(set)
      words = words.split(" ")
      tags = tags.split(" ")
      total = ''
      tag = ''
      for idx, relation in enumerate(tags):
        print(relation)
        d[relation].add(idx)
      res = ""
      for i, s in d.items():
        print(s)
        temp = []
        for idx in s:
          w = okt.pos(words[idx])
          print("word",w)
          if  w == "Noun" or w == "Adjective":
            temp.append(words[idx])
        total += ' '.join(temp)
        total += "➜"
        tag += i
        tag += "➜"
      return total[:-1], tag[:-1]

### POPUP for relation 

In [369]:
analyzed = '''
'Speaker Speaker Message': '모든 인간은 평등하다고'
'Affected_party Affected_party Affected_party Affected_party Manner': '모든 인간을 모든 측면에서 똑같이'
'Means Means Means Goal Goal Goal Goal Goal Goal Goal Goal Goal Goal Goal Goal Goal': '평등에 대한 요구는 모든 불평등을 악으로 보는 것이 아니라 충분한 이유가 제시되지 않은 불평등을 제거하는 데
'Message Message Message Message Message Message Message Time Time Time Time Message Message Message': "'이유 없는 차별 금지'라는 조건적 평등 원칙은 차별 대우를 할 때는 이유를 제시할 것을"
'Agent Agent Agent Act Act': '특정한 사람들에게는  차별적 대우를
'Term Speaker Speaker Speaker': '사람들을 특정한 부류로 구분하는'
'Item Item Item Item Item Item Item Category Category Category Category Category Category': '근대의 여러 인권 선언에 나타난 평등 개념은 개인들 사이의 평등성을 타고난 자연적 권리로'
'Entity Event Event Event Event Event Event Event': '롤스는 정의에 따라 규칙을 준수하는 것만으로는 정의를 담보할'
'Entity Entity Event Event Event Event Event Event Event Event': '그 규칙이 더 높은 도덕적 권위를 지닌 다른 이념과 충돌할'
'Support Proposition Proposition Proposition Proposition Proposition Proposition Proposition Proposition': "그는 평등한 대우를 받기 위한 영역 성질로서 '도덕적 인격'을"
'''

In [373]:
s = [i for i in analyzed.split("\n") if i]
tag=[]
word=[]
popup = ""
for e in s:
  k = e.split(': ')
  tags = k[0].replace("'","").replace('"',"")
  words = k[1].replace("'","").replace('"',"")
  print("tags", tags)
  print()
  print("words", words)
  tag.append(tags)
  word.append(words)
for i in range(len(word)):
  d = defaultdict(set)
  words = word[i].split(" ")
  tags = tag[i].split(" ")
  total = ""
  for idx, relation in enumerate(tags):
    d[relation].add(idx)
  res = ""
  t = ""
  for i, s in d.items():
    t += str(i)
    t += " ➜ "
    temp = []
    for idx in s:
      w = okt.pos(words[idx])
      print("OKT ",w)
      for _ in w:
        print("_ ",_)
        if  _[1] != "Josa":
          temp.append(_[0])
    total += ' '.join(temp)
    total += " ➜ "
  t = t[:-2]
  total = total[:-2]
  print(t, total)
  target = "<a data-toggle='tooltip' data-placement='top' title='"+ t + "'>" + total +"</a><br>"
  popup += "\n\n"+target

tags Speaker Speaker Message

words 모든 인간은 평등하다고
tags Affected_party Affected_party Affected_party Affected_party Manner

words 모든 인간을 모든 측면에서 똑같이
tags Means Means Means Goal Goal Goal Goal Goal Goal Goal Goal Goal Goal Goal Goal Goal

words 평등에 대한 요구는 모든 불평등을 악으로 보는 것이 아니라 충분한 이유가 제시되지 않은 불평등을 제거하는 데
tags Message Message Message Message Message Message Message Time Time Time Time Message Message Message

words 이유 없는 차별 금지라는 조건적 평등 원칙은 차별 대우를 할 때는 이유를 제시할 것을
tags Agent Agent Agent Act Act

words 특정한 사람들에게는  차별적 대우를
tags Term Speaker Speaker Speaker

words 사람들을 특정한 부류로 구분하는
tags Item Item Item Item Item Item Item Category Category Category Category Category Category

words 근대의 여러 인권 선언에 나타난 평등 개념은 개인들 사이의 평등성을 타고난 자연적 권리로
tags Entity Event Event Event Event Event Event Event

words 롤스는 정의에 따라 규칙을 준수하는 것만으로는 정의를 담보할
tags Entity Entity Event Event Event Event Event Event Event Event

words 그 규칙이 더 높은 도덕적 권위를 지닌 다른 이념과 충돌할
tags Support Proposition Proposition Proposition Proposition Propos

In [372]:
def main():
    h= Highlight(text)
    tagged = h.highlight() # 하이라이트까지 구현된 상태
    paragraphs = [i for i in tagged.split('\n') if i]
    counts = len(paragraphs)
    print("Length of paragraphs ",len(paragraphs))
    total = ''
    popup = ''
    rel = Relation()
    for idx in range(counts):
      total += "\n"
      paragraph = kss.split_sentences(paragraphs[idx])
      summarizer = Summarize(text, idx)
      txt, id = summarizer.ensembleSummarize()
      print("summarized ",txt)
      sent_counts = len(paragraph)
      original, q = rel.relationPopup(txt)
      print("ORIGINAL ", original)
      for sent in range(sent_counts):
        relation, relation_string = rel.relationPopup(paragraph[sent])
        print("RELATION ", relation, "relation_string ",relation_string)
        p = "<a data-toggle='tooltip' data-placement='top' title="+ str(relation) + ">" + str(relation_string) +"</a>"
        popup += p
        if txt[:3] in paragraph[sent]:
          print("일치함 !")
          string ="<u data-toggle='tooltip' data-placement='top' title= "+ str(relation) + " style='text-decoration:underline; text-decoration-color:#906fa8; font-weight: bold; text-decoration-style: wavy'>" + paragraph[sent] +"</u>"
          total += string
        else:
          total += paragraph[sent]
        
    print(total)
    with open("/content/result.txt",'w') as file:
        file.write(total)

    with open("/content/popup.txt",'w') as file:
        file.write(popup)

In [374]:
with open("/content/popup.txt",'w') as file:
    file.write(popup)